# 1. Agent bricks: Review Apspect Extraction Agent

This notebook walks through instructions to build an Information Extraction Agent in Agent Bricks to extract aspect insights & sentiments from raw reviews

Data Flow: 
**raw reviews -> review aspect extractions**  -> location aspect daily -> flag all issues -> issue diagnosis and recommendations

## Build Information Extraction Agent

Extract Structured Insights from Raw Reviews

Example: 

- Information Extraction Agent: Schema Config [`ie_agent_config.json`](resource_configs/ie_agent_config.json)
- Instructions: 

```
1. Extract ALL relevant metadata mentioned or implied in the review.
   - Include fields such as: star rating, review date, length of stay, and overall sentiment.
   - If metadata is missing, set to null — do not guess or invent.

2. Extract EVERY relevant aspect from the review text.
   - Use the predefined aspect list (Arrival & Departure, Staff & Service, In-Room Experience, Food & Beverage, Facilities & Amenities, Environment & Location, Value & Loyalty).
   - For each aspect:
       • Identify sentiment (very_positive, positive, neutral, negative, very_negative).
       • Include short, verbatim evidence (phrases directly from the review).
       • opinion_terms: array of short polarity-bearing words/phrases tied to this aspect (e.g., “spotless,” “friendly,” “overpriced,” “noisy AC”); use verbatim spans when possible
   - Deduplicate aspects — each aspect should appear at most once.
   - Do not miss subtle mentions, mixed opinions, or multiple details for the same aspect.
   - Capture both positive and negative details accurately, without omitting context.

3. Extract all **entities** explicitly or implicitly mentioned in the review.
   - Entities include: staff roles or names, attractions, nearby locations etc.
   - Keep entity names consistent and distinct.
   - Do not fabricate entities; if unclear, set to null.
   - Avoid redundancy: each unique entity should appear only once.

4. Output clean, valid JSON following the specified schema (no extra text, no commentary).
   - Ensure consistency across reviews.
   ```

## Batch Inference With IE Agent Endpoint and `AI_QUERY`
#### [IE Query in SQL Editor](queries/AI Query Extraction with KIE.dbquery.ipynb)


In [0]:
%sql
CREATE OR REPLACE TABLE lakehouse_inn_catalog.voc.review_extractions AS
WITH query_results AS (
  SELECT
    review_id,
    review_text,
    ai_query(
      'kie-b59e4876-endpoint',
      review_text,
      failOnError => false
    ) AS respon
    
    se
  FROM (
    SELECT review_id, review_text
    FROM lakehouse_inn_catalog.voc.raw_reviews
  )
)
SELECT
  review_id,
  review_text,
  response.result AS response,
  response.errorMessage AS error
FROM query_results;
